# KerasTuner

KerasTuner is an easy-to-use, scalable hyperparameter optimization framework that solves the pain points of hyperparameter search. KerasTuner comes with Bayesian Optimization, Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment with new search algorithms.
In this notebook we show how to use the **KerasTuner** for automatic network optimization (and, in general, hyperparameter tuning). This example uses the breast cancer dataset which we have already seen in the course and is completely self contained. However if you want to further understand what's going on please refer to:

* [Official website](https://keras.io/keras_tuner/)
* [Tutorial at Tensorflow](https://www.tensorflow.org/tutorials/keras/keras_tuner)
* [Tutorial at medium.com](https://haneulkim.medium.com/hyperparameter-tuning-with-keras-tuner-full-tutorial-f8128397e857)

# Setup(s)

## Standard libraries setup

In [1]:
#very common libraries, that we for sure are using
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## Talos setup

In [2]:
#making sure KerasTuner is installed.
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt

## Seed setup

In [4]:
#resetting the seeds
!wget -O support_code.py https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/lab_day1/support_code.py
%run support_code.py
n1 = 0
reset_random_seeds(n1)

--2025-10-30 12:41:53--  https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/lab_day1/support_code.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6242 (6.1K) [text/plain]
Saving to: ‘support_code.py’

support_code.py     100%[===================>]   6.10K  --.-KB/s    in 0s      

2025-10-30 12:41:53 (85.3 MB/s) - ‘support_code.py’ saved [6242/6242]

Import all libraries: yes
importing libraries
Defining functions
DONE!


## Data setup

In [5]:
#libraries for this block
import sklearn.datasets
from sklearn.model_selection import StratifiedShuffleSplit

# loading data
from sklearn.datasets import load_breast_cancer
bcancer = load_breast_cancer()
y = bcancer.target
X = pd.DataFrame(bcancer.data, columns=bcancer.feature_names)

# normalizing
X = (X - X.mean())/X.std()

In [6]:
#declaring a sss object
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

#sss.split() returns two iterables over the two pieces of data
for train_index, val_index in sss.split(X=X, y=y):
  x_train = X.iloc[train_index, :]
  x_val   = X.iloc[val_index, :]

  y_train = y[train_index]
  y_val   = y[val_index]

# KerasTuner workflow

## Define the (hyper)model

When you build a model for the tuning of the hyperparameters, you also define the **hyperparameter search space** in addition to the model architecture.
The model you set up for fine-tuning is called a **hypermodel**.

With **KerasTuner**, you can define a hypermodel through two approaches:

- by using a **model builder function**
- by subclassing the **HyperModel class** of the KerasTuner API

Here we use a model builder function to define the classification model.
The model builder function returns a compiled model and uses the hyperparameters you define to finetune the model:

- n. of units in the first dense layer
- learning rate

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(30,))) ## n. of features in the BreastCancer dataset

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 16-128
  hp_units = hp.Int('units', min_value=16, max_value=128, step=16)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Hyperparamaters to be explored

The Keras Tuner has four tuners available: i) RandomSearch, ii) Hyperband, iii) BayesianOptimization, iv) Sklearn.
In this tutorial, you use the `Hyperband tuner`.

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (`max_epochs`).

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3, ## factor: Integer, the reduction factor for the number of epochs and number of models for each bracket. Defaults to 3.
                     directory='my_dir',
                     project_name='intro_to_kt')

Reloading Tuner from my_dir/intro_to_kt/tuner0.json


Create a callback to stop training early after reaching a certain value for the validation loss.



In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early], overwrite=True,)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 331ms/step - accuracy: 0.3940 - loss: 1.9809 - val_accuracy: 0.9231 - val_loss: 0.6860
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9353 - loss: 0.5353 - val_accuracy: 0.9121 - val_loss: 0.3132
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9501 - loss: 0.2437 - val_accuracy: 0.9341 - val_loss: 0.2081
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9668 - loss: 0.1592 - val_accuracy: 0.9341 - val_loss: 0.1616
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9798 - loss: 0.1217 - val_accuracy: 0.9451 - val_loss: 0.1360
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9798 - loss: 0.1004 - val_accuracy: 0.9780 - val_loss: 0.1192
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9879 - loss: 0.0868 - val_accuracy: 0.9780 - val_loss: 0.1065
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9879 - loss: 0.0771 - val_accuracy: 0.9890 - 

In [27]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - accuracy: 0.7049 - loss: 1.0283 - val_accuracy: 0.9670 - val_loss: 0.0722
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9884 - loss: 0.0625 - val_accuracy: 0.9890 - val_loss: 0.0557
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9866 - loss: 0.0540 - val_accuracy: 0.9451 - val_loss: 0.0813
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9896 - loss: 0.0440 - val_accuracy: 0.9780 - val_loss: 0.0485
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9896 - loss: 0.0284 - val_accuracy: 0.9670 - val_loss: 0.0471


In [28]:
eval_result = hypermodel.evaluate(x_val, y_val)
print("[test loss, test accuracy]:", eval_result)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.9647 - loss: 0.1129
[test loss, test accuracy]: [0.14574988186359406, 0.9561403393745422]


# Further steps

The code above is a **very** minimal example and works as a starting point. Stuff to consider:

* each combination of hyperparameter is trained once, with a 70/30 default split. Using `.evaluate_models()` it's possible to do a proper k-fold crossvalidation (see [scan documentation](https://autonomio.github.io/talos/#/Scan), search "evaluate_models")
* the default approach of trying all the combinations can become unfeasible very quickly. The `Scan` function supports several policies for sampling a subset of the hyperparameter space. See the [Towardsdatascience's tutorial](https://towardsdatascience.com/tune-the-hyperparameters-of-your-deep-learning-networks-in-python-using-keras-and-talos-2a2a38c5ac31) for a more in-depth example